<a href="https://colab.research.google.com/github/harithaharry001/CovidDataset/blob/master/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Dropout, MaxPooling2D, BatchNormalization, Activation, Flatten, Dense


In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from os import listdir
from matplotlib import image
from skimage.transform import resize


def load_data(path):
    
    train_files = []
    for foldername1 in listdir(path):
        filepath1 = path  + "/" + foldername1
        for filename1 in listdir(filepath1):
            train_files.append(filepath1 + "/" + filename1)
            
    # Original Dimensions
    image_width = 224
    image_height = 224
    channels = 3
    
    loaded_images = np.ndarray(shape=(len(train_files), image_height, image_width, channels),dtype=np.float32)
    print(type(loaded_images))
    loaded_class = []
    i = 0
    for foldername in listdir(path):
        filepath = path  + "/" + foldername
        print("Folder : ",filepath)
        for filename in listdir(filepath):
            # load image
            img_data = image.imread(filepath + "/" + filename)
            
            # store loaded image
            img_data = resize(img_data, (224, 224, 3))
            loaded_images[i] = img_data
            loaded_class.append(foldername)
            i = i + 1
            #print('> loaded %s %s' % (filename, img_data.shape))
        print('Loaded: ',i , ' images from ',filepath)
            
    return loaded_images,loaded_class


In [5]:
path = r"/content/drive/MyDrive/Dataset"
x,y = load_data(path)
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

<class 'numpy.ndarray'>
Folder :  /content/drive/MyDrive/Dataset/Covid
Loaded:  478  images from  /content/drive/MyDrive/Dataset/Covid
Folder :  /content/drive/MyDrive/Dataset/Normal
Loaded:  2358  images from  /content/drive/MyDrive/Dataset/Normal


In [7]:
print(train_x.shape)
print(test_x.shape)

(1650, 224, 224, 3)
(708, 224, 224, 3)


In [8]:
# Creating a Sequential model
model = Sequential()

# 1st Convolution Layer
model.add(Conv2D(filters=96, kernel_size=(11,11), input_shape=(224, 224, 3), strides=(4,4), padding='valid'))
# Normalization
model.add(BatchNormalization())
# Activation Function
model.add(Activation('relu'))
# Max-Pooling
model.add(MaxPooling2D((3,3), strides=(2,2), padding='valid'))


# 2nd Convolution Layer
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same'))
# Normalization
model.add(BatchNormalization())
# Activation Function
model.add(Activation('relu'))
# Max-Pooling
model.add(MaxPooling2D((3,3), strides=(2,2), padding='valid'))


# 3rd Convolution Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), padding='same'))
# Normalization
model.add(BatchNormalization())
# Activation Function
model.add(Activation('relu'))


# 4th Convolution Layer
model.add(Conv2D(filters=384, kernel_size=(3,3), padding='same'))
# Normalization
model.add(BatchNormalization())
# Activation Function
model.add(Activation('relu'))


# 5th Convolution Layer
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
# Normalization
model.add(BatchNormalization())
# Activation Function
model.add(Activation('relu'))
# Max-Pooling
model.add(MaxPooling2D((3,3), strides=(2,2), padding='valid'))


# Flattening before passing to the Dense layer
model.add(Flatten())


# 1st Dense Layer
model.add(Dense(4096))
# Dropout
model.add(Dropout(0.4))
# Normalization
model.add(BatchNormalization())
# Activation Function
model.add(Activation('relu'))


# 2nd Dense Layer
model.add(Dense(4096))
# Dropout
model.add(Dropout(0.4))
# Normalization
model.add(BatchNormalization())
# Activation Function
model.add(Activation('relu'))


# Output softmax Layer
model.add(Dense(2))
# Activation Function
model.add(Activation('softmax'))



model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 54, 54, 96)        34944     
_________________________________________________________________
batch_normalization_7 (Batch (None, 54, 54, 96)        384       
_________________________________________________________________
activation_8 (Activation)    (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 26, 26, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 26, 26, 256)       614656    
_________________________________________________________________
batch_normalization_8 (Batch (None, 26, 26, 256)       1024      
_________________________________________________________________
activation_9 (Activation)    (None, 26, 26, 256)      

In [9]:
import pandas as pd
from keras.utils import to_categorical
y_train_values, unique = pd.factorize(train_y)
print('y_train ', y_train_values, unique)


y_test_values, unique = pd.factorize(test_y)
print('y_test ', y_test_values, unique)

y_train_one_hot = to_categorical(y_train_values)
y_test_one_hot = to_categorical(y_test_values)


y_train  [0 0 0 ... 0 1 0] ['Normal' 'Covid']
y_test  [0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1
 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0
 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0
 1 0 1 0 1 1 1 0 1 1 1 0 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0
 0 1 0 1 0 0 0 1 1 0 0 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0
 1 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0

In [10]:
x_train_normalization = train_x / 255.0
x_test_normalization = test_x / 255.0

In [11]:
from sklearn.utils import shuffle


x_shuffled_default, y_shuffled_default = shuffle(x_train_normalization, y_train_one_hot)

In [12]:
model.compile(
loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x_shuffled_default, y_shuffled_default, epochs = 30, batch_size = 32)